# Operational Wind Lidar Processing with LROSE

## About NSF NCAR Integrated Sounding System (ISS)

The [Integrated Sounding System](https://www.eol.ucar.edu/observing_facilities/iss) is a suite of instruments for profiling the atmosphere from ground level. Instrumentation we operate includes wind profiling radars, scanning wind lidars, sounding systems, and more. The ISS is a requestable facility that deploys for field projects around the world.

<figure align="center">
<img src="../images/lidar_tutorial_images/ISS_DG_pano_labelled.png"/>
<figcaption align = "center"> A labeled view of instruments at an ISS site on the island of Diego Garcia during the DYNAMO field campaign. </figcaption>
</figure>

### Scanning wind lidar
The ISS currently operates a [Leosphere WindCube WLS300S](https://www.vaisala.com/sites/default/files/documents/WEA-MET-WindCube-Scan-Lidar-Produ ct-Spotlight-B212058EN-A.pdf) scanning wind lidar, primarily used to generate wind profiles in the lowest few kilometers of atmosphere. 

<figure align="center">
<img src="../images/lidar_tutorial_images/iss_cfact_windcube.png"/>
<figcaption align = "center"> Installing the ISS WindCube lidar at the CFACT campaign in Heber City, Utah. </figcaption>
</figure>

### Operational data products
<figure align="center">
<img src="../images/lidar_tutorial_images/lrose_ams_tutorial_dataflow.drawio.png"/>
<figcaption align = "center"> Diagram of ISS lidar operational dataflow. </figcaption>
</figure>

We perform basic data processing in near real time during field campaigns, so users can verify that the lidar is functioning correctly and scientists can look at initial data collected. Our automated processing pipeline consists of four steps:
1. Convert from Leosphere netCDF format to CFRadial netCDF
2. Create single-scan plot files from CFRadial netCDF
3. Calculate and plot VAD winds from CFRadial netCDF RHI scans
4. Calculate and plot consensus averages from the VAD winds

## Set up

In [3]:
# python imports
import os
import glob
import datetime

# environment variables
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['PARAMS_DIR'] =  os.environ['BASE_DIR'] + '/params/lidar'
os.environ['LIDAR_RAW_DIR'] = '/home/jovyan/share/data_files_for_notebooks/m2hats/20230820'

## Sample data

This tutorial uses one day's worth of lidar data from the [M2HATS](https://www.eol.ucar.edu/field_projects/m2hats) field campaign in summer of 2023. During this project, the WindCube lidar was used mainly to derive wind profiles, so it ran PPI scans almost continuously. It also ran N-S and E-W RHI scans, horizon sector scans, and vertical stares once hourly.

The WindCube lidar produces data in netCDF format, but it's not quite CFRadial compliant. It produces one file for each scan, and identifies the scan by type (PPI, RHI, fixed, or DBS) and by scan ID, which identifies the specific parameters (azimuth or elevation, gate spacing, etc) associated with the scan. In this data, the PPI scans with scan ID 178 were designed to be used for VAD wind calculations and have an elevation of 35º.

In [ ]:
!ls ${LIDAR_RAW}/*.nc | head 20

## Convert Leosphere format files to CFRadial

The first step in our dataflow is to convert files to CFRadial format. This means that all subsequent processing steps are run on CFRadial data, which avoids the need for further instrument-specific processing. 

We use RadxConvert to convert to CFRadial. RadxConvert can read and convert Leosphere format netCDFs, and in the future will also be able to read and convert data from HALO lidars.

In [ ]:

# set up output directory
os.environ['LIDAR_CFRAD_DIR'] = os.environ['BASE_DIR'] + '/data/lidar/cfradial'
!rm -rf $LIDAR_CFRAD_DIR
!mkdir -p $LIDAR_CFRAD_DIR

# select input files
lidar_raw = os.environ['LIDAR_RAW']
files = glob.glob(lidar_raw + '/*.nc')

# convert with RadxConvert
for f in files[0:5]:
    !RadxConvert -f $f -outdir $CFRADIAL_DIR -params ${PARAMS_DIR}/RadxConvert.params
# this could maybe just be -f $RAW_DIR/*? I think it can do multiple at once
    
    # what special values for parameters do we use - would be nice to have a way to find non-default parameters

#/opt/iss-system/lrose/bin/RadxConvert -f /data/iss/lotos2024/iss2/lidar/rawdata/20241003/WLS200s-181_2024-10-03_20-26-40_rhi_41_50m.nc -outdir /data/iss/lotos2024/iss2/lidar/cfradial -params /iss/lrose/params/RadxConvert.params

## Generate plots for each CFRadial PPI or RHI scan

### HawkEye

Operationally we use HawkEye from LROSE for this purpose. HawkEye is generally run interactively with a user interface, but it's possible to run non-interactively by supplying a virtual frame buffer such as Xvfb. 

On ISS field servers, we start Xvfb with:

 `/usr/bin/Xvfb :1000.0 -screen 0 2000x2000x24`. 

 This can be run manually, or can be set up as a systemd service to start automatically. With Xvfb running, we set the `$DISPLAY` variable when calling HawkEye so it will use the virtual frame buffer: 
 
 `env DISPLAY=:1000.0 HawkEye -params ...`

We don't currently have a solution for running HawkEye in a jupyter notebook, because of its GUI needs. Below are two example plots that we generated using HawkEye locally on an ISS server. We generally create a plot of velocity and a plot of relative beta for each lidar scan.

<figure align="center">
<img src="../images/lidar_tutorial_images/hawkeye_velocity.png"/>
<figcaption align = "center"> An example HawkEye plot of lidar velocity. </figcaption>
</figure>

<figure align="center">
<img src="../images/lidar_tutorial_images/hawkeye_beta.png"/>
<figcaption align = "center"> An example HawkEye plot of lidar relative beta. </figcaption>
</figure>

### PyArt

* how to do this for tutorial purposes only

In [ ]:
# can I get XVFB working in a notebook
#python xvfb wrapper maybe?

## `iss-lidar` python package

We created our own python package to do VAD calculations and consensus averaging on wind lidar data. That code is on [github](https://github.com/ncar/iss-lidar) and newly available as a python package on [PyPi](https://pypi.org/project/iss-lidar/). This code is very much a work in progress, so please feel free to get in touch with questions, issues, comments, or pull requests.

- link to repo
- why we wrote it (why our own vad?)
- brief acknowledgements

### Scripts
Mention scripts that can be used if you download the repo? Like ppi_to_vad and vad_to_consensus, if you don't want it to be part of your own code. This tutorial is a notebook so we'll be calling functions directly, but you can use the scripts too. Maybe include the help output from each script here. Are these included in the package install? If so how do you use em?

In [ ]:
# make sure iss-lidar package is up to date
!pip install --upgrade iss-lidar

from iss_lidar import VADSet
from iss_lidar import ConsensusSet
from iss_lidar import create_cns_filename

## Generate VAD winds for each RHI file

To get a vertical profile of 3-dimensional wind speed and direction, the iss-lidar package uses the velocity-azimuth-display (VAD) algorithm. We follow the implementation as described in [Newsom et al. 2019](https://doi.org/10.2172/1238069).

The iss-lidar code reads in CFRadial PPI scans and produces a daily netCDF file containing VAD winds from all the PPI scans from that day. We use the format included in Appendix A of Newsom et al. 2019 for our VAD winds (and consensus winds) output.

### Thresholding explanation
.....

In [ ]:
# set up output directory
os.environ['LIDAR_VAD_DIR'] = os.environ['BASE_DIR'] + '/data/lidar/vad'
!rm -rf $LIDAR_VAD_DIR
!mkdir -p $LIDAR_VAD_DIR

# select input files
cfrad_dir = os.environ['LIDAR_CFRAD_DIR']
ppis_for_vad = glob.glob(cfrad_dir + '/*ppi_178*') # fix this glob for cfrad file names
print(ppis_for_vad, type(ppis_for_vad)) #is this just a list i can pass to from_PPIs directly?

# set minimum CNR threshold
min_cnr = -22

# create VADSet object from PPI files
vadset = VADSet.from_PPIs(ppis_for_vad, min_cnr)

# perform optional thresholding
vadset.load_thresholds(args.threshold_config)
vadset.apply_thresholds()
        
# save output to netCDF
save(vadset, os.environ['LIDAR_VAD_DIR']) # does this need to be an environ var or can it just be a python variable?!?

# break this cell up into multiples to further explain each of the commands?
# plots ?!?!?


## Generate consensus averaged winds from VAD winds

To create 30-minute averaged winds from VAD winds, the iss-lidar package uses consensus averaging.

The iss-lidar code can either read in a VAD netCDF file, or use a VADSet object in memory (i.e. read in and calculated from PPIs). It outputs daily netCDF files of 30-minute consensus averages.

* documentation on consensus averaging?

In [ ]:
# set up output directory 
os.environ['LIDAR_CNS_DIR'] = os.environ['BASE_DIR'] + '/data/lidar/consensus'
!rm -rf $LIDAR_CNS_DIR
!mkdir -p $LIDAR_CNS_DIR

# does this have to be an environ too or can it just be a python variable

# consensus window
cns_window = 5 # why did we pick this??
timespan = datetime.timedelta(minutes=30) #30 minute averages

# this can be done from VADSet object directly:

# or it can be done by reading in VAD data from file:
vadfile =# hardcode path? or get it from glob?
vs = VADSet.from_file(args.vadfile)
cs = ConsensusSet.from_VADSet(vs, cns_window, timespan)

# populate filename with consensus averaging timespan
fpath = create_cns_filename(timespan, cs.stime[0], os.environ.get['LIDAR_CNS_DIR'])

# save file
cs.to_ARM_netcdf(fpath)

if (args.plot):
    plot(fpath, cs.u, cs.v, cs.stime, cs.height)

Vertical stare plot examples

Not yet publically available code, but we're working on it :(

Mention scripts that can be used if you download the repo? Like ppi_to_vad and vad_to_consensus, if you don't want it to be part of your own code. 